In [97]:
import pickle
import os
from datetime import datetime
import pandas as pd
import numpy as np

In [98]:
model_path = r"C:\data\Models\xgboost_regressor_r2_0_92_v1.pkl"
model = pickle.load(open(model_path,"rb"))

In [99]:
model.predict([[0.00536201,  0.25987344, -0.47082647, -0.23611548,  0.14321374,
        2.09651259, -0.16966066,  0.        ,  0.10829126, -0.58950634,
        0.        ,  0.        , -0.57735027,  1.73205081, -0.57735027,
       -0.22941573]])

array([364.6361], dtype=float32)

# Load SC

In [100]:
sc_dump_path = r"C:\data\Models\sc.pkl"
sc = pickle.load(open(sc_dump_path, "rb"))

In [101]:
sc

StandardScaler()

# Users Input

In [141]:
Date = "10/06/24"
Hour = 12
Temperature = 34
Humidity = 55
Visibility = 100
sku = 10016283
Wind_speed = 67
Rainfall = 0.7
Snowfall = 0.2

# Converting User Data into Model Consumable Format

In [103]:
from datetime import datetime
def get_string_to_datetime(date):
    dt = datetime.strptime(Date, "%d/%m/%y")
    return {"day": dt.day, "month": dt.month, "year": dt.year, "weekday": dt.strftime("%A")}
get_string_to_datetime(Date)
str_to_date = get_string_to_datetime(Date)
str_to_date

{'day': 10, 'month': 6, 'year': 2024, 'weekday': 'Monday'}

In [129]:
# Ensure str_to_date is a dictionary with "day", "month", and "year" keys
str_to_date = {"day": 1, "month": 7, "year": 2024}

# Define your input list properly (replace with actual values or variables)
u_input_list = [
    Hour,  # Replace with actual value or variable
    Temperature,  # Replace with actual value or variable
    Humidity,  # Replace with actual value or variable
    Wind_speed,  # Replace with actual value or variable
    Visibility,
    Solar_Radiation,
    Rainfall,  # Replace with actual value or variable
    Snowfall,  # Replace with actual value or variable  # Replace with actual value or variable
    sku,  # Replace with actual value or variable
    str_to_date["day"], 
    str_to_date["month"], 
    str_to_date["year"],
]

# Define your features name list properly
features_name = [
    'Hour', 
    'Temperature(°C)', 
    'Humidity(%)',
    'Wind speed (m/s)', 
    'Visibility (10m)',
    'Solar Radiation (MJ/m2)',
    'Rainfall(mm)', 
    'Snowfall (cm)',
    'sku', 
    'Day', 
    'Month', 
    'Year'
]

# Create DataFrame
data_u_input = pd.DataFrame([u_input_list], columns=features_name)
data_u_input


,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),sku,Day,Month,Year
0,12,34,55,67,100,20.0,0.7,0.2,10017413,1,7,2024


In [130]:
u_input_list

[12, 34, 55, 67, 100, 20.0, 0.7, 0.2, 10017413, 1, 7, 2024]

In [131]:
features_name

['Hour',
 'Temperature(°C)',
 'Humidity(%)',
 'Wind speed (m/s)',
 'Visibility (10m)',
 'Solar Radiation (MJ/m2)',
 'Rainfall(mm)',
 'Snowfall (cm)',
 'sku',
 'Day',
 'Month',
 'Year']

In [132]:
# Example assuming 'weekday' is a key in your str_to_date dictionary
str_to_date = {
    'wee_day': 'Monday'  # Example value, adjust as per your actual data
}

def days_data(week_day):
    days_name = ['Monday', 'Saturday', 'Sunday', 'Tuesday']
    days_name_data = np.zeros((1, len(days_name)), dtype="int")
    data_days = pd.DataFrame(days_name_data, columns=days_name)
    if week_day in days_name:
        data_days[week_day] = 1
    return data_days

data_days = days_data(str_to_date["wee_day"])
data_days




,Monday,Saturday,Sunday,Tuesday
0,1,0,0,0


In [133]:
data_for_pred = pd.concat([data_u_input,data_days], axis=1)
data_for_pred

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),sku,Day,Month,Year,Monday,Saturday,Sunday,Tuesday
0,12,34,55,67,100,20.0,0.7,0.2,10017413,1,7,2024,1,0,0,0


In [63]:
['Hour', 'Temperature(°C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Solar Radiation (MJ/m2)',
       'Rainfall(mm)', 'Snowfall (cm)', 'sku', 'Day', 'Month', 'Year',
       'Monday', 'Saturday', 'Sunday', 'Tuesday']

['Hour',
 'Temperature(°C)',
 'Humidity(%)',
 'Wind speed (m/s)',
 'Visibility (10m)',
 'Solar Radiation (MJ/m2)',
 'Rainfall(mm)',
 'Snowfall (cm)',
 'sku',
 'Day',
 'Month',
 'Year',
 'Monday',
 'Saturday',
 'Sunday',
 'Tuesday']

In [134]:
pd.concat([data_u_input, data_days], axis=1)

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),sku,Day,Month,Year,Monday,Saturday,Sunday,Tuesday
0,12,34,55,67,100,20.0,0.7,0.2,10017413,1,7,2024,1,0,0,0


In [135]:
sc_data_for_pred = sc.transform(data_for_pred)
sc_data_for_pred


array([[ 0.14834893,  7.29206528, -0.20650284, 56.00033769, -1.70383868,
        52.92916755,  2.14765567,  0.2       ,  0.11978994, -1.43165827,
        -5.        ,  7.        ,  1.73205081, -0.57735027, -0.57735027,
        -0.22941573]])

# Prediction in Deployment

In [142]:
model.predict(sc_data_for_pred)

print(f"Trend Demand on date: {Date}, and Time: {Hour} is : {model.predict(sc_data_for_pred)}")

Trend Demand on date: 10/06/24, and Time: 12 is : [429.06296]
